In [ ]:
import os
import sys
import glob

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import seaborn as sns
# sns.set(font_scale = 1.5)
sns.set_style("white")
import pandas as pd
import datetime
%matplotlib inline

In [ ]:
# connecting to google drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from google.colab import drive
drive.mount('/content/drive')

# directory where the data, figures etc will be saved:
save_dir = '/content/drive/Shareddrives/TuriLab/Data/PTSD_project/!analysis/saveFolder/'

Mounted at /content/drive


#### Loading DATA

In [ ]:
worksheet = gc.open('all_cohorts_freezing_FINAL').sheet1

# get_all_values gives a list of rows.
data = worksheet.get_all_values()
# print(data)

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(data)
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.reset_index(inplace=True, drop=True)

# swapping out 'ptsd' to sefl in condition column
df.loc[:,('condition')].replace('ptsd','sefl', inplace=True)
# replacing NA to NaN
df.replace('NA', np.NaN, inplace=True)
# changing `freezing` dtype to float
df['freezing'] = pd.to_numeric(df['freezing'],
                                        downcast='float')
# adding unique ID-s based on `id` and `cohort`
df['cohort_id']= df['cohort']+'_'+df['id']

df['date_time'] = df['date']+' '+df['time']
# df['man_date_time'] = pd.to_datetime(df['date_time'])

df['date'] = df['date'].astype('datetime64[ns]', copy=False)

print(df.dtypes)
print('=================')
df.info()
# TODO add DOB-s
# calculate time after SEFLA column

0
freezing                    float32
genotype                     object
id                           object
day                          object
condition                    object
sex                          object
cohort                       object
exposure_mins                object
analyzed_duration            object
date                 datetime64[ns]
time                         object
els                          object
cohort_id                    object
date_time                    object
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1165 entries, 0 to 1164
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   freezing           1160 non-null   float32       
 1   genotype           1165 non-null   object        
 2   id                 1165 non-null   object        
 3   day                1165 non-null   object        
 4   condition          1165 non-null   obj

#### Loading some more DATA with DOB-s

In [ ]:
worksheet = gc.open('id_dob').sheet1

# get_all_values gives a list of rows.
data = worksheet.get_all_values()
# print(data)

# Convert to a DataFrame and render.
import pandas as pd
dob_df = pd.DataFrame.from_records(data)
dob_df.columns = dob_df.iloc[0]
dob_df.drop(dob_df.index[0], inplace=True)
dob_df.reset_index(inplace=True, drop=True)

dob_df['cohort_id']= dob_df['cohort']+'_'+dob_df['id']
dob_df.drop(['id', 'cohort'], axis=1, inplace=True )
dob_df['dob'] = dob_df['dob'].astype('datetime64[ns]')

dob_df.head()
print(f'dob_df shape{dob_df.shape}')
dob_df.info()

dob_df shape(147, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   dob        147 non-null    datetime64[ns]
 1   cohort_id  147 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 2.4+ KB


#### Merging the freezing and DOB

In [ ]:
# merging freezing w/ DOBs
merged_df = df.merge(dob_df,left_on='cohort_id', right_on='cohort_id')
# print(merged_df.dtypes)
print(f'merged_df shape {merged_df.shape}')
merged_df.info()

merged_df shape (1165, 15)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1165 entries, 0 to 1164
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   freezing           1160 non-null   float32       
 1   genotype           1165 non-null   object        
 2   id                 1165 non-null   object        
 3   day                1165 non-null   object        
 4   condition          1165 non-null   object        
 5   sex                1165 non-null   object        
 6   cohort             1165 non-null   object        
 7   exposure_mins      1160 non-null   object        
 8   analyzed_duration  1160 non-null   object        
 9   date               1096 non-null   datetime64[ns]
 10  time               1096 non-null   object        
 11  els                1165 non-null   object        
 12  cohort_id          1165 non-null   object        
 13  date_time          1096 non-null   o

In [ ]:
df_sefla = merged_df[merged_df["day"] == "sefla"]

df_sefla['age_at_SEFLA'] = (df_sefla.loc[:,('date')] - 
                            df_sefla.loc[:,('dob')]).dt.days/7
df_sefla.reset_index(inplace=True, drop=True)

els_sefla_age = df_sefla[df_sefla['els'] == 'y']
age = els_sefla_age['age_at_SEFLA'].mean()
print(f'Age of els mice at sefl A (in weeks): {age}')

Age of els mice at sefl A (in weeks): 17.066666666666666


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


#### Createing a table for age ELS mice and age matched controls

In [ ]:
df = merged_df
# age_matched_m = df[(df['age_at_SEFLA']>=17) & (df['els'] =='n')]
# mice.reset_index(inplace=True)
# age_matched_m['cohort_id'].unique()

age_matched_controls = ['ptsd1_sert26a-3', 'ptsd1_sert26a-2', 'ptsd3_2', 'ptsd3_12',
       'ptsd3_1', 'ptsd3_3', 'ptsd3_11', 'ptsd5_11', 'ptsd5_61',
       'ptsd5_12', 'ptsd5_13', 'ptsd5_64', 'ptsd5_62']

els_mice = df[df['cohort_id'].str.startswith('els1')]
print(f'els mice: {els_mice.shape}')
  
age_matched_mice = df[df['cohort_id'].isin(age_matched_controls)]
print(f'age matched mice: {age_matched_mice.shape}')

els_age_matched = pd.concat([els_mice, age_matched_mice], axis=0)
print(f'els and age matched: {els_age_matched.shape}')

els mice: (336, 15)
age matched mice: (112, 15)
els and age matched: (448, 15)


In [ ]:
##########################
# saving the data to csv #
##########################

# els_age_matched.to_csv(os.path.join(save_dir, 'els_mice_age_matched_controls.csv'))

In [ ]:
# number of els vs control
els_age_matched.groupby('els')['cohort_id'].nunique()

els
n    25
y    30
Name: cohort_id, dtype: int64